In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import pandas as pd
from google.colab import files  # for CSV download

# --- Constants ---
BASE_URL = "https://data.apps.fao.org/gismgr/api/v2/catalog/workspaces/WAPOR-3/mapsets"
TUNISIA_BBOX = [7.0, 38.0, 12.0, 29.5]  # [minX, maxY, maxX, minY]
PIXEL_AREA = 100 * 100  # 100 m × 100 m = 10,000 m²

# --- Helper function to paginate API ---
def collect(url, fields=["code", "downloadUrl"]):
    out, data = [], {"links": [{"rel": "next", "href": url}]}
    while "next" in [x["rel"] for x in data["links"]]:
        url_ = [x["href"] for x in data["links"] if x["rel"] == "next"][0]
        r = requests.get(url_)
        r.raise_for_status()
        data = r.json()["response"]
        out += [tuple(x.get(y) for y in fields) for x in data["items"]]
    return sorted(out)

# --- Function to extract mean raster over Tunisia ---
def extract_mean_raster(url):
    opt = gdal.TranslateOptions(projWin=TUNISIA_BBOX, bandList=[1], unscale=True)
    ds = gdal.Translate("/vsimem/temp.tif", f"/vsicurl/{url}", options=opt)
    arr = ds.GetRasterBand(1).ReadAsArray().astype(float)
    arr[arr <= 0] = np.nan
    return arr

# --- Fetch rasters ---
aeti_rasters = collect(f"{BASE_URL}/L2-AETI-A/rasters")
tbp_rasters = collect(f"{BASE_URL}/L2-TBP-A/rasters")

# --- Aquastat Cr data (rainfed fraction) ---
aquastat_cr = {
    2018: 0.6354496208,
    2019: 0.6355362946900001,
    2020: 0.5872156013000001,
    2022: 0.5872156013000001
}
# Compute average Cr for all years we have
avg_cr = np.mean(list(aquastat_cr.values()))

# --- Compute annual water productivity ---
records = []

GVAa = 6.7e9  # USD/year
p = 0.35      # USD per kg biomass

for (et_code, et_url), (tbp_code, tbp_url) in zip(aeti_rasters, tbp_rasters):
    year = int(et_code[-4:])
    
    ET_arr = extract_mean_raster(et_url)
    TBP_arr = extract_mean_raster(tbp_url)
    
    ET_mean = np.nanmean(ET_arr)
    TBP_mean = np.nanmean(TBP_arr)
    
    V_ETb = np.nansum(ET_arr * PIXEL_AREA)
    
    # Use average Cr from Aquastat
    Awp1 = GVAa * (1 - avg_cr) / V_ETb
    
    WPb = np.nanmean(TBP_arr / (ET_arr + 1e-6))
    Awp2 = WPb * p
    
    records.append({
        "year": year,
        "ET_mean": ET_mean,
        "TBP_mean": TBP_mean,
        "V_ETb": V_ETb,
        "Awp1": Awp1,
        "Awp2": Awp2
    })

# --- Convert to DataFrame ---
df = pd.DataFrame(records).sort_values("year")
print(df)

# --- Plot Awp1 and Awp2 ---
plt.figure(figsize=(8,5))
plt.plot(df["year"], df["Awp1"], marker='o', label="Awp1 (USD/m³)")
plt.plot(df["year"], df["Awp2"], marker='s', label="Awp2 (USD/m³)")
plt.xlabel("Year")
plt.ylabel("Water Productivity")
plt.title("Annual Water Productivity in Tunisia (Adjusted)")
plt.legend()
plt.grid(True)
plt.show()

# --- Save CSV and download in Colab ---
csv_filename = "tunisia_water_productivity.csv"
df.to_csv(csv_filename, index=False)
files.download(csv_filename)
